In [ ]:
import geopandas as gpd
import os
import fiona

In [ ]:
data_pkg_path = '../data'
filename = 'karnataka.gpkg'
path = os.path.join(data_pkg_path, filename)

In [ ]:
import fiona
layers = fiona.listlayers(path)
print(layers)

In [ ]:
roads_gdf = gpd.read_file(path, layer='karnataka_major_roads')
filtered = roads_gdf[roads_gdf['ref'].str.match('^NH') == True]
roads_reprojected = filtered.to_crs('EPSG:32643')
roads_reprojected['length'] = roads_reprojected['geometry'].length
total_length = roads_reprojected['length'].sum()

In [ ]:
districts_gdf = gpd.read_file(path, layer='karnataka_districts')
districts_reprojected = districts_gdf.to_crs('EPSG:32643')

In [ ]:
joined = gpd.sjoin(roads_reprojected, districts_reprojected, how='left', op='intersects')
results = joined.groupby('DISTRICT')['length'].sum()/1000

Convert the results to round number

In [ ]:
# You could also use results.astype(int)
results = results.round()

In [ ]:
output_filename = 'national_highways_by_districts.csv'
output_dir = '../output'
output_path = os.path.join(output_dir, output_filename)
results.to_csv(output_path)
print('Successfully written output file at {}'.format(output_path))